# Środowisko Google Kubernetes Engine 

Logowanie kontem github. 

## Wykorzystanie Google Cloud Storage


#### Projekt
* Wszystki dane przynależą do konkretnego projektu.
* Do projektu mogą mieć dostęp użytkownicy. 
* Projekt ma zdefiniowane metody uwierzytelniające, rozliczenia, monitorowanie etc. 
#### Kubełek (bucket)
* Kubełek (buckket) to kontener na pliki/obiekty. 
* Nazwa Bucketu musi być unikalna w skali całej usługi u wszystkich użytkowników (!) 
* Kubełków nie można zagnieżdzać
* W kubełkach możemy tworzyć foldery i tam logicznie grupować pliki.
* Kubełek wraz z zawartością może zostać udostępniony publicznie.
* Kubełkowi nie można zmienić nazwy lub metadanych. Trzeba go usunąć i stworzyć ponownie.
#### Obiekt 
* obiekty przechowywane w kubełkach 
* obiekty mają zawartość oraz metadane
* obiekty są niemodyfikowalne 

Do operacji na Google Storage można wykorzystać narzędzie `gsutil`: 

##### Operacje na kubełkach
* listowanie kubełków (buckets) - `ls`
* tworzenie nowego kubełka - `mb`
* usuwania kubełka - `rm`
* listowanie zawartości kubełków - `ls` 
* udostępnianie - `iam`

##### Operacje na obiektach
* dodawania pliku do kubełka - `cp`
* kopiowanie między kubełkami - `cp`
* usuniecie z kubełka - `cp`
* pobranie informacji o obiekcie - `stat`


#### Operacje na kubełkach

In [ ]:
! gsutil ls

In [ ]:
! echo $USER  # wyswietlenie nazwy uzytkownika

In [ ]:
! gsutil mb gs://bucket-$USER  # stworzenie bucketu

In [ ]:
! gsutil ls -L -b gs://bucket-$JUPYTERHUB_USER # listowanie zawartości 

In [ ]:
! gsutil du -s  gs://bucket-$USER # ile zajmuje przestrzeni?

#### Operacje na zawartości kubełków

In [ ]:
! gsutil ls -r gs://bucket-$USER/ # listowanie zawartosci kubełka

In [ ]:
! gsutil cp ~/work/git/ds-notebooks/README.md gs://bucket-$USER  # upload obiektu do kubełka

In [ ]:
! gsutil ls -r gs://bucket-$USER # listowanie zawartości

In [ ]:
! gsutil iam get gs://bucket-$USER

In [ ]:
! gsutil stat gs://bucket-$USER/README.md # metadane obiekty w kubełku

In [ ]:
! gsutil iam ch allUsers:objectViewer gs://bucket-$USER # dodanie uprawnien do odczytu

Po wykonaniu tego polecenia nasz kubełek staje się publiczny i możemy się do niego 
http://storage.googleapis.com/bucket-NAZWA_UZYTKOWNIKA. Zawartość pliku można odczytać poprzez http://storage.googleapis.com/bucket-NAZWA_UZYTKOWNIKA/NAZWA_OBIEKTU

In [ ]:
! gsutil iam ch -d allUsers:objectViewer gs://bucket-$USER # usuniecie uprawnien do odczytu

Teraz można ponownie zweryfikowac możliwość publicznego odczytu danych z kubełka.

#### Przeniesienie do docelowego kubełka

In [ ]:
%%bash
gsutil cp -r gs://bucket-$USER gs://DOCELOWY_BUCKET
gsutil rm -r gs://bucket-$USER

### ***** Zadanie 4 *****
a) Utwórz plik zawierający aktualną date i godzinę. 
b) Udostępnij plik publicznie na google storage. 

## Apache Spark na GKE

Podobnie jak na klastrze nawiążemy połączenie z sesją Spark i odczytamy dane z pliku udostępnionego na GCS.

In [ ]:
# popbieramy nazwę użytkownika do zmiennej python'owej
import os
user_name = os.environ.get('USER')
print(user_name)

Pobieramy dane ze strony narodów zjednoczonych o populacji na świecie:
http://data.un.org/    

In [ ]:
#! wget https://archive.ics.uci.edu/ml/machine-learning-databases/00368/Facebook_metrics.zip
! wget http://data.un.org/_Docs/SYB/CSV/SYB63_1_202009_Population,%20Surface%20Area%20and%20Density.csv  

Zmieńmy nazwę

In [ ]:
! mv 'SYB63_1_202009_Population, Surface Area and Density.csv'  pop.csv

Podejrzyjmy dane:

In [ ]:
! head pop.csv

Plik zawiera dwie linie nagłówka, czyli jedną trzeba usunąć. 

In [ ]:
! tail -n +2  pop.csv > pop2.csv
! head pop2.csv

Teraz lepiej.

Następnie przenosimy dane do naszego kubełka, żeby móc je odczytać za pomocą Spark'a

In [ ]:
! gsutil cp pop2.csv   gs://bucket-$USER/
! gsutil ls gs://bucket-$USER/


Ustawiamy zmienną python'ową 'path', która wskazuje na plik 'pop2.csv' w kubełku.

In [ ]:
path="gs://bucket-" + user_name + "/pop2.csv" 
path

Uruchamiamy sesję Spark'ową na GKE

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
.config("spark.executor.instances", "1") \
.config("spark.executor.memory", "1g") \
.getOrCreate()

# Dla porównania tak to wyglądało na klastrze:

#from pyspark.sql import SparkSession
#spark = SparkSession \
#.builder \
#.master('yarn-client') \
#.config('spark.driver.memory','1g') \
#.config('spark.executor.memory', '1g') \
#.config('spark.ui.port',f'{ui_port}') \
#.appName(f'ds-{user_name}') \
#.getOrCreate()

Wczytujemy dane

In [ ]:
df = spark.read.load(path, format="csv", sep=",", inferSchema="true", header="true")

Jakiego typu jest df?

In [ ]:
type (df)

In [ ]:
df.printSchema()

Sprawdźmy najświeższe dane z Polski, czyli jak wygląda populacja w roku 2020:

In [ ]:
df.filter("Year=2020 and _c1='Poland'").select("Series","_c1", "Value").show(truncate=False)

### ***** Zadanie 5 *****

a) Jaka była suma populacji Polski i Niemiec w kolejnych raportowanych latach?

b) Oblicz średnią populację we wszystkich raportowanych latach dla kolejnych reginów oraz podaj 10 najbardziej licznych.

In [ ]:
# Tu wpisz kod do Zadania 5

In [ ]:
spark.stop()